# Retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

Interface:
- Input: A Query (string)
- Output: A list of documents (standardized LangChain Document objects)

Common retrievers include:
- Vector store retrievers
- Search api retrievers
- Relational database retrievers


In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

### Loading Documents

In [ ]:
loaders = [
    TextLoader("data/langchain.md"),
    TextLoader("data/langchain2.md"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

## Retrieving Documents

**Conflicting needs in document retrieval:**

- Need for small chunks to maintain embedding accuracy
- Need for longer chunks to preserve context

Steps:
1. Split and store small chunks of data.
2.	The retriever first fetches the small chunks.
3.	It then looks up the parent IDs for those chunks.
4.	Finally, it returns the larger documents.

In [ ]:
# Define a text splitter that will be used to create child documents from larger parent documents.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# Initialize a vector store named "full_documents" which will index the child chunks of the documents.
# The OllamaEmbeddings model "snowflake-arctic-embed:33m" is used to generate embeddings for these chunks.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# Set up an in-memory storage layer that will store the parent documents.
store = InMemoryStore()

# Create a retriever that uses the previously defined vector store, document store, and child splitter.
# This retriever will be able to fetch relevant parent documents based on queries and split them into child chunks as needed.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [ ]:
retriever.add_documents(docs, ids=None)
list(store.yield_keys())

In [ ]:
sub_docs = vectorstore.similarity_search("What is LangChian", k=1)
print(sub_docs)

In [ ]:
retrieved_docs = retriever.invoke("What is LangChian")
print(len(retrieved_docs[0].page_content))
print(retrieved_docs)

## Retrieving Large Chunks

In [ ]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# The storage layer for the parent documents
store = InMemoryStore()

### ParentDocumentRetriever
    - Splits and stores small chunks for embedding/indexing
    - During retrieval, fetches small chunks first
    - Then looks up and returns the parent documents of those chunks

In [ ]:
# Create a retriever that uses the previously defined vector store, document store, child splitter, and parent splitter.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
# Add documents to the retriever
retriever.add_documents(docs)

# Get the total number of keys in the store
len(list(store.yield_keys()))

In [ ]:
sub_docs = vectorstore.similarity_search("what is LangChain used for", k=5)

print(sub_docs)

In [ ]:
retrieved_docs = retriever.invoke("what is LangChain used for")

print(len(retrieved_docs[0].page_content))
print(retrieved_docs[0].page_content)

## Putting it all together 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2:1b')

In [ ]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
print(prompt)

In [ ]:
# Function to format documents by joining their content
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt  # Apply the prompt template
        | model  # Use the language model to generate a response
        | StrOutputParser()  # Parse the output string
)

print(chain.invoke("What is LangChain"))